In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import time
from datetime import datetime
import xmltodict as xtd
import xml.etree.cElementTree as ET
import zillow

# API Keys
from config import gkey, zkey, zws_id

In [2]:
# target_city = input("Please input the city where you would like to move : ")
# radius=int(input("Please input the radius around the city : "))

target_city = 'atlanta'

params = {
    "address": target_city,
    "key": gkey
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Run request
res = requests.get(base_url, params=params).json()

# Convert to JSON
# target_geo = response.json()
# pprint(target_geo)

# Extract lat/lng
lat = res["results"][0]["geometry"]["location"]["lat"]
lng = res["results"][0]["geometry"]["location"]["lng"]
state = res["results"][0]["address_components"][2]["short_name"]

z_params = {
    "latlng":f"{lat},{lng}",
    "result_type":"postal_code",
    "key":gkey
}

pcode_query = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json", params=z_params).json()
zip_code = pcode_query['results'][0]['address_components'][0]['long_name']
# Print results
print(f"city:   {target_city}\nlatlng: {lat}, {lng}\nstate:  {state}\nzip:    {zip_code}")

city:   atlanta
latlng: 33.7489954, -84.3879824
state:  GA
zip:    30334


In [3]:
params = {
    "input": "target_city",
    "inputtype":"textquery",
    "key": gkey
}

url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

In [4]:
params = {
    "state":"ga",
    "city":"atlanta",
    "childtype":"neighborhood",
    "zws-id":zws_id
}

url = "http://www.zillow.com/webservice/GetRegionChildren.htm"
res = requests.get(url, params=params)

In [5]:
doc = xtd.parse(res.text)

In [6]:
df = pd.DataFrame(doc['RegionChildren:regionchildren']['response']['list']['region'])
df.head()

,id,name,zindex,url,latitude,longitude
0,269381,Midtown,"{'@currency': 'USD', '#text': '216900'}",http://www.zillow.com/local-info/GA-Atlanta/Mi...,33.786845,-84.379061
1,269306,Downtown,"{'@currency': 'USD', '#text': '149100'}",http://www.zillow.com/local-info/GA-Atlanta/Do...,33.758149,-84.391189
2,269397,Old Fourth Ward,"{'@currency': 'USD', '#text': '165400'}",http://www.zillow.com/local-info/GA-Atlanta/Ol...,33.762616,-84.37087
3,269391,North Buckhead,"{'@currency': 'USD', '#text': '445800'}",http://www.zillow.com/local-info/GA-Atlanta/No...,33.862514,-84.370276
4,274592,Morningside - Lenox Park,"{'@currency': 'USD', '#text': '639900'}",http://www.zillow.com/local-info/GA-Atlanta/Mo...,33.79931,-84.354281


In [28]:
api = zillow.ValuationApi()
zpid="2100641621"
detail_data = api.GetZEstimate(zws_id, zpid)
address = detail_data.full_address.get_dict()
address['zipcode']
# adf = pd.DataFrame(address, index=[0])
# adf

'90292'

In [16]:
zdf = detail_data.get_dict()
pd.DataFrame(zdf).head()

,zpid,similarity_score,links,full_address,zestimate,local_realestate,extended_data
amount,2100641621,None,NaN,NaN,2760217,NaN,NaN
amount_change_30days,2100641621,None,NaN,NaN,-25841,NaN,NaN
amount_currency,2100641621,None,NaN,NaN,USD,NaN,NaN
amount_last_updated,2100641621,None,NaN,NaN,10/06/2018,NaN,NaN
bathrooms,2100641621,None,NaN,NaN,NaN,NaN,None


In [18]:
address = "3400 Pacific Ave., Marina Del Rey, CA"
postal_code = "90292"

data = api.GetDeepSearchResults(zws_id, address, postal_code)

In [29]:
dsr = data.get_dict()
pd.DataFrame(dsr).head()

,zpid,similarity_score,links,full_address,zestimate,local_realestate,extended_data
amount,2100641621,None,NaN,NaN,2760217,NaN,NaN
amount_change_30days,2100641621,None,NaN,NaN,-25841,NaN,NaN
amount_currency,2100641621,None,NaN,NaN,USD,NaN,NaN
amount_last_updated,2100641621,None,NaN,NaN,10/06/2018,NaN,NaN
bathrooms,2100641621,None,NaN,NaN,NaN,NaN,3.0
